In [1]:
!pip install -U transformers
# Install below if using GPU
!pip install accelerate

   ---------------------------------------- 0.0/10.0 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.0 MB ? eta -:--:--
   -------- ------------------------------- 2.1/10.0 MB 6.9 MB/s eta 0:00:02
   ----------------- ---------------------- 4.5/10.0 MB 8.7 MB/s eta 0:00:01
   ---------------------------- ----------- 7.1/10.0 MB 9.9 MB/s eta 0:00:01
   -------------------------------------- - 9.7/10.0 MB 10.6 MB/s eta 0:00:01
   ---------------------------------------- 10.0/10.0 MB 10.4 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.2
    Uninstalling transformers-4.45.2:
      Successfully uninstalled transformers-4.45.2


In [2]:
import os
import pandas as pd
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
#import dotenv
from tqdm import tqdm
from langchain_community.vectorstores import Chroma
from langchain.schema import Document
import matplotlib.pyplot as plt

## To run Hugging Face OpenSource models
# Needs to manually install Visual C++ Tools from: https://visualstudio.microsoft.com/visual-cpp-build-tools/
from InstructorEmbedding import INSTRUCTOR
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import Chroma

#For some reason, "context" cant be used as input variable, it should be named as "summaries"
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate
import warnings
from rich import print
import random, re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download necessary NLTK data (run this only once)
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

# Suppress all warnings
warnings.filterwarnings("ignore")

c:\ProgramData\anaconda3\envs\rag\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\os190\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\os190\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\os190\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
import torch
import torch.nn as nn

# Check if CUDA is available
print(f"CUDA Available: {torch.cuda.is_available()}")

# Print CUDA device name
if torch.cuda.is_available():
    print(f"Device Name: {torch.cuda.get_device_name(0)}")

CUDA Available: True

Device Name: NVIDIA GeForce RTX 4060 Laptop GPU

In [4]:
# Ensure GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda

### Hyperparameters for this test:

In [5]:
index_test_name = 'TestA'
LLM_model = 'google/flan-t5-large'
temperature_parameter=0.25
retriever_matches_k = 3

In [6]:
# Define directories
# Define the persistent directory containing the VectorDB
script_dir =  os.getcwd()
persistent_dir = os.path.abspath(os.path.join(script_dir,'..' ,'index', index_test_name))

# SE REQUIERE UTILIZAR EL MISMO MODELO DE SENTENCE-TRANSFORMER CON EL QUE SE CREO EL INDEX, sin ello se enfrentaran:
#Espacios Vectoriales Diferentes:
#
#Cada modelo de sentence-transformer genera embeddings en su propio espacio vectorial. Estos espacios son definidos por los pesos y arquitecturas específicas de cada modelo.
#Los embeddings generados por el modelo A no son directamente comparables con los del modelo B porque están en espacios diferentes y no alineados.
#Incompatibilidad de Embeddings:
#
#Las representaciones vectoriales (embeddings) de las mismas frases pueden ser muy distintas entre modelos. Por ejemplo, una oración podría tener un vector [0.1, 0.2, 0.3] en el modelo A y [0.5, -0.1, 0.2] en el modelo B.
#Esto significa que medidas de similitud como la similitud coseno no serán significativas, ya que los vectores no están en el mismo espacio.
#Resultados Incorrectos o Sin Sentido:
#
#Al realizar consultas, las comparaciones entre embeddings del índice (modelo A) y los embeddings de la consulta (modelo B) producirán resultados erróneos.
#Podrías obtener altas similitudes entre oraciones no relacionadas o bajas similitudes entre oraciones muy similares.
#Pérdida de Precisión y Rendimiento:
#
#El rendimiento del sistema de recuperación de información se degradará significativamente.
#Los usuarios recibirán resultados irrelevantes, lo que afecta la usabilidad y confiabilidad del sistema.
#Consistencia en Procesamiento de Lenguaje Natural:
#
#Los modelos pueden tener diferentes enfoques para manejar ciertos aspectos del lenguaje, como negaciones, sarcasmo o lenguaje coloquial.
#Esto añade otra capa de inconsistencia entre los embeddings generados por diferentes modelos.

embed_model = "sentence-transformers/all-mpnet-base-v2"


# Para correr las predicciones en GPU
model_kwargs = {'device': 'cuda:0'}  # specify GPU device
encode_kwargs = {'normalize_embeddings': True}

### Set a Q&A LLM Chain to provide context and system prompts on every query
This will run peristantly, consulting already stored vectors

In [7]:
##### Embedding model (sentence-transformer)
model_name = embed_model
model_kwargs = {'device': 'cuda:0'}  # specify GPU device
encode_kwargs = {'normalize_embeddings': True}
hf_embed_model = HuggingFaceInstructEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

##### LLM model
# loading the Llama3 model from local
'''
llm_model = OllamaLLM(model=LLM_model,
                temperature=temperature_parameter,
                num_thread=8,
                )
'''
import requests
from langchain_core.runnables import Runnable

class LocalLLM(Runnable):
    def __init__(self, url):
        self.url = url

    def _run(self, inputs):
        # Expecting 'question' and 'summaries' in inputs
        prompt = inputs[0]
        
        response = requests.post(
            f"{self.url}/generate", 
            json={"prompt": prompt.text}
        )
        response.raise_for_status()
        return response.json()["generated_text"]  # Adjust based on the API response structure

    async def _arun(self, inputs):
        prompt = inputs['prompt']
        max_length = inputs.get('max_length', 50)
        
        response = requests.post(
            f"{self.url}/generate", 
            json={"prompt": prompt, "max_length": max_length}
        )
        response.raise_for_status()
        return response.json()[0]["generated_text"]  # Adjust based on the API response structure

    def invoke(self, *args, **kwargs):
        """ Synchronously invoke the _run method. """
        # If using additional parameters, pass them to _run
        return self._run(args)

    async def ainvoke(self, *args, **kwargs):
        """ Asynchronously invoke the _arun method. """
        # If using additional parameters, pass them to _arun
        return await self._arun(kwargs)

# Set up the Local LLM with the URL of your Flask/FastAPI server
llm_model = LocalLLM(url="http://localhost:5000")  # Adjust the URL if needed
# loading the vectorstore
vectorstore = Chroma(persist_directory=persistent_dir, embedding_function=hf_embed_model)
# casting  the vectorstore as the retriever, taking the best 3 similarities
retriever = vectorstore.as_retriever(search_kwargs={"k":retriever_matches_k})

# formating the docs
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

template = """you must respond only with the hs_code from the metadata.


context:
{summaries}

Question:
{question}
"""

# Define the LLM chain (using the Llama3.1 model)
llm_chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm_model,
    chain_type='stuff',
    retriever=retriever,
    return_source_documents=True,  # To get both the answer and source docs
    chain_type_kwargs={
            "prompt": PromptTemplate(
                template=template,
                #For some reason, "context" cant be used as input variable, it should be named as "summaries"
                input_variables=["question", "summaries"],
            ),
        },
)

context = """As a logistics shipping arrival inspector, your primary responsibility is to inspect incoming shipments and accurately classify goods 
using the Harmonized System (HS) code based on the descriptions provided in the shipping manifests. You will thoroughly review the manifest details, 
including product type, material composition, function, and intended use, to determine the correct HS code. 

Your task is to:
Carefully read and analyze the product descriptions from the manifest.
Identify key characteristics of the goods, such as 
type (e.g., electronics, textiles, machinery), 
material (e.g., plastic, metal, organic), 
and usage (e.g., household, industrial, medical).
Use your knowledge of the HS code classification system to assign the most appropriate HS code for each product based on its description.
Ensure compliance with international trade regulations by selecting precise codes to avoid delays or penalties.
Remember to be thorough and accurate in your classification, as this impacts customs processing, tariffs, and legal requirements."""



load INSTRUCTOR_Transformer
max_seq_length  512


In [8]:
## Test
#query = "sodium lignosulphonate"
query = "kilogram organic coffee bean certified fairtrade utz roasted usa global distribution"

# Execute the chain with the query
#For some reason, "context" cant be used as input variable, it should be named as "summaries"
result = llm_chain({"question": query, "summaries": context})

print(f"[bold yellow]The requested item's description to search HTS code is:[/bold yellow]\n{query}")
print(f"[bold green]The response of the LLM is:[/bold green]\n{result['answer']}")


#print("The documents sent to the LLM are:")
#for i in range(len(result["source_documents"])):
#    print(result["source_documents"][i].page_content)
#    print(result["source_documents"][i].metadata)
#     
#results = vectorstore.similarity_search_with_score(query=query, k=retriever_matches_k)
#print("Top n coincidences from the index are:")
#for doc, score in results:
#    print(f"Document content: {doc.page_content}, Code: {doc.metadata},Similarity Score: {score}")
#
#
## Find the document with the highest score
#max_doc, max_score = max(results, key=lambda x: x[1])
#print("Best match from the index is:")
#print(f"Document content: {max_doc.page_content}, Code: {max_doc.metadata}, Similarity Score: {max_score}")



The requested item's description to search HTS code is:
kilogram organic coffee bean certified fairtrade utz roasted usa global distribution

The response of the LLM is:
004

### Run 50 iterations of 30 random samples each

In [9]:
# Load the CSV file
df = pd.read_csv('../../data validation verified.csv', encoding='utf-8', dtype=str)

# Extract the queries from the 'Raw_data_input' column
all_queries = df[['Raw_data_input', 'Expected_output','Expected_output_two_digits', 'Expected_output_four_digits']].dropna()  # Ensure no missing values
#all_queries = df[['Validation_input', 'Expected_output']].dropna()  # Ensure no missing values


# Create an empty list to store the results
results = []

# Clean text function to avoid garbage
def clean_text(text):
    text = text.lower() # Convert text to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text) # Remove non-alphanumeric characters (keep spaces)
    words = word_tokenize(text)  # Tokenization
    words = [word for word in words if word not in merged_stopwords]     # Remove stopwords
    # Lemmatization to convert the received sentence to a meaningful sentence
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(word) for word in words]
    # Join words back into a sentence
    meaningful_text = ' '.join(words)
    return meaningful_text 


# Definir stopwords en español y las stopwords adicionales específicas del contexto
stop_words_es = set(stopwords.words('spanish'))
stop_words_en = set(stopwords.words('english'))
additional_stopwords = {
    'hs', 'code', 'hscode', 'hs-code', 'hs  code', 'pallets', 'plts', 'shipper', 'declares', 'hs code',
    'containing', 'contains', 'meter', 'cubic', 'packages', 'load', 'loaded', 'weight', 
    'netweight', 'kg', 'kgs', 'cb', 'cbm', 'goods', 'parts', 'pieces', 'accessories', 'packing', 
    'declared', 'dangerous', 'impression', 'items', 'sheets', 'codes', 
    'sin', 'impresion', 'containers', 'pc', 'abv', 'net', 'gross', 'cif', 'aduana', 'customs', 
    'value', 'tax', 'duty', 'freight', 'port', 'terminal', 'consignee', 'consignor', 'invoice', 
    'manifest', 'quantity', 'description', 'volume', 'packaging', 'shipment', 'delivery', 'origin', 
    'destination', 'transport', 'carrier', 'export', 'import', 'tariff', 'item', 'declaration', 
    'clearance', 'documentation', 'commercial', 'charge', 'fees', 'logistics', 'shipping', 
    'container', 'unit', 'measurement', 'certification', 'palletized', 'metric', 'commodity', 
    'classification', 'entry', 'exportation', 'importation', 'bonded', 'zone', 'trade', 'license', 'bottle', 'bottles', 'cl',
    'ancho', 'largo', 'mm', 'pcs', 'xhc', 'stc', 'uks','x','k', 'pty', 'id', 'cp', 'ncm', 'ne', 'itpa', 'zz', 'xg', 'topmag',
    'rtmx', 'fcl', 'cf','f', 'xdc', 'pkgs'
}

# Combinar stopwords estándar con stopwords del contexto específico
merged_stopwords = stop_words_es.union(stop_words_en).union(additional_stopwords)

## Save the DataFrame to a new CSV file
#output_file_path = '../Prediction_vs_Expected_TestB.csv'
#predictions_df.to_csv(output_file_path, index=False)
#
#print(f"Results saved to {output_file_path}")



In [10]:
def run_predictions(descriptions_to_predict):
    # Select 30 random queries
    # Shuffle the original DataFrame
    df_shuffled = all_queries.sample(frac=1).reset_index(drop=True)
    selected_queries = df_shuffled.sample(descriptions_to_predict)

    # Now, process each query with llm_chain
    #print("Predicting HS codes of random Validation Data rows...")
    for index, row in selected_queries.iterrows():
        query = clean_text(row['Raw_data_input'])
        expected_output = row['Expected_output']

        # Execute the LLM chain (using a mock prediction for illustration here)
        result = llm_chain({"question": query, "summaries": context})  # This is the actual call
        llm_prediction = result['answer']  # Assuming LLM returns a numeric 6-digit value as the prediction

        # Append the result in the required structure
        results.append({
            "Raw_data_input": row['Raw_data_input'],
            "Raw_data_input_processed": query,
            "LLM_prediction": llm_prediction,  # Here, the prediction from LLM
            "Expected_output": expected_output,
            "Expected_output_two_digits": row['Expected_output_two_digits'],
            "Predicted_output_two_digits": llm_prediction[:2],
            "CorrectMatch_two_digits": 1 if llm_prediction[:2] == row['Expected_output_two_digits'] else 0 ,
            "Expected_output_four_digits": row['Expected_output_four_digits'],
            "Predicted_output_four_digits": llm_prediction[:4],
            "CorrectMatch_four_digits": 1 if llm_prediction[:4] == row['Expected_output_four_digits'] else 0 
        })

    # Convert the results to a DataFrame
    predictions_df = pd.DataFrame(results)

    return predictions_df

# Function to calculate accuracy (both for 2 and 4 digits)
def calculate_accuracy(iteration_df):
    correct_two_digits = 0
    correct_four_digits = 0
    total = len(iteration_df)

    for index, row in iteration_df.iterrows():
        best_hs_code = str(row['LLM_prediction'])[:4]
        hs_actual = str(row['Expected_output'])[:4]

        # Check 2-digit accuracy
        if best_hs_code[:2] == hs_actual[:2]:
            correct_two_digits += 1

        # Check 4-digit accuracy
        if best_hs_code == hs_actual:
            correct_four_digits += 1

    accuracy_at_two_digits = correct_two_digits / total if total > 0 else 0
    accuracy_at_four_digits = correct_four_digits / total if total > 0 else 0

    return accuracy_at_two_digits, accuracy_at_four_digits

n_iterations = 50
n_predictions = 30
accuracy_results_list = []

for iteration in tqdm(range(n_iterations), desc="Running MonteCarlo simulation"):
    #print("Running iteration #:" , iteration+1)
    iteration_df = run_predictions(n_predictions)
    # Calculate accuracy
    acc_two_digits, acc_four_digits = calculate_accuracy(iteration_df)

    # Store the results
    accuracy_results_list.append([iteration + 1, acc_two_digits, acc_four_digits])
    #print("Accuracy of this iteration: ", acc_two_digits, acc_four_digits)

# Convert results to DataFrame for further analysis
accuracy_results_df = pd.DataFrame(accuracy_results_list, columns=['Iteration', 'Accuracy_At_Two_Digits', 'Accuracy_At_Four_Digits'])
accuracy_results_df

plt.figure(figsize=(10, 6))
accuracy_results_df[['Accuracy_At_Two_Digits', 'Accuracy_At_Four_Digits']].boxplot()
plt.title('Monte Carlo Simulation: Accuracy at Two and Four Digits')
plt.ylabel('Accuracy')
plt.show()


Running MonteCarlo simulation:   0%|          | 0/50 [00:04<?, ?it/s]


KeyboardInterrupt: 